# Ordinary differential equation model with the vector field defined in Zig
Simon Frost (@sdwfrost), 2024-06-03

## Introduction

While Julia is a high-level language, it is possible to define the vector field for an ordinary differential equation (ODE) in another language and call it from Julia. This can be useful for performance reasons (if the calculation of the vector field in Julia happens to be slow for some reason), or if the vector field is already defined, for example, in another codebase. Julia's `ccall` makes it easy to call a compiled function in a shared library created by a language that supports the generation of C-compatible shared libraries, such as Zig.

## Libraries

In [ ]:
using OrdinaryDiffEq
using Libdl
using Plots
using BenchmarkTools

## Transitions

We define the vector field in Zig; it is easiest for this function to be in-place, so that we do not have to do any memory management on the Zig side. This approach is also more efficient, as it reduces the number of allocations needed.

In [ ]:
Zig_code = """
export fn sir_ode(du: [*c]f64, u: [*c]const f64, p: [*c]const f64, t: [*c]const f64) void {
    const beta: f64 = p[0];
    const c: f64 = p[1];
    const gamma: f64 = p[2];
    const S: f64 = u[0];
    const I: f64 = u[1];
    const R: f64 = u[2];
    const N: f64 = S + I + R;
    _ = t;

    du[0] = -beta * c * S * I / N;
    du[1] = beta * c * S * I / N - gamma * I;
    du[2] = gamma * I;
}
""";

We then compile the code into a shared library.

In [ ]:
const Ziglib = tempname();
open(Ziglib * "." * "zig", "w") do f
    write(f, Zig_code)
end
run(`zig build-lib -dynamic -O ReleaseSafe -fPIC -femit-bin=$(Ziglib * "." * Libdl.dlext) $(Ziglib * "." * "zig")`);

We can then define the ODE function in Julia, which calls the Zig function using `ccall`. `du`, `u`, `p` are arrays of `Float64`, which are passed using pointers. `t` is passed as a `Ref` pointer to a `Float64` value.

In [ ]:
function sir_ode_jl!(du,u,p,t)
    ccall((:sir_ode,Ziglib,), Cvoid,
          (Ptr{Float64}, Ptr{Float64}, Ptr{Float64}, Ptr{Float64}), du, u, p, Ref(t))
end;

## Time domain and parameters

In [ ]:
δt = 0.1
tmax = 40.0
tspan = (0.0,tmax)
u0 = [990.0,10.0,0.0] # S,I,R
p = [0.05,10.0,0.25]; # β,c,γ

## Solving the ODE

In [ ]:
prob_ode = ODEProblem{true}(sir_ode_jl!, u0, tspan, p)
sol_ode = solve(prob_ode, Tsit5(), dt = δt);

## Plotting

In [ ]:
plot(sol_ode)

## Benchmarking

In [ ]:
@benchmark solve(prob_ode, Tsit5(), dt = δt)

We can compare the performance of the Zig-based ODE with the Julia-based ODE.

In [ ]:
function sir_ode!(du,u,p,t)
    (S,I,R) = u
    (β,c,γ) = p
    N = S+I+R
    @inbounds begin
        du[1] = -β*c*I/N*S
        du[2] = β*c*I/N*S - γ*I
        du[3] = γ*I
    end
    nothing
end
prob_ode2 = ODEProblem(sir_ode!, u0, tspan, p)
sol_ode2 = solve(prob_ode2, Tsit5(), dt = δt)
@benchmark solve(prob_ode2, Tsit5(), dt = δt)

Note that the performance of the Zig-based vector field is similar to the one defined in Julia.